## 论坛文本数据处理

In [1]:
import pandas as pd
import numpy as np
import re
import jieba
from zhon.hanzi import punctuation
import warnings
warnings.filterwarnings('ignore')

In [47]:
# Import the data
info = pd.read_csv('../data0629/info_immigration.csv',encoding='utf-8-sig')
# cmt = pd.read_csv('../data0629/cmt_immigration.csv',encoding='utf-8-sig')

In [48]:
def re_content(text):
    if text == ' ':
        return ' '
    # remove the system info '[...]'
    new_content = re.sub(r'\[[\n][^\]]*[\n]\]','',text)
    # remove the \n
    new_content = new_content.replace('\n',' ')
    # remove the url
    new_content = re.sub(r'http[s]?://[^\s]*','',new_content)
    # remove the .jpg/.jpeg
    new_content = re.sub(r'[\w]*\.[a-zA-Z]{3,4}','',new_content)
    # remove the (xxx.xx KB) or (xx.xx MB)
    new_content = re.sub(r'\([\d]*\.[\d]* [a-zA-Z]{2}\)','',new_content)
    
    # 97 is a special case
    new_content = new_content.replace('97','九七')
    # remove the timestamp
    new_content = re.sub(r'\d{2,4}-\d{1,2}-\d{1,2} \d{2}:\d{2}:\d{2}','',new_content)
    new_content = re.sub(r'\d{2,4}-\d{1,2}-\d{1,2} \d{2}:\d{2}','',new_content)
    new_content = re.sub(r'\d{2,4}-\d{1,2}-\d{1,2}','',new_content)
    new_content = re.sub(r'\d{2,4}年\d{1,2}月\d{1,2}日','',new_content)
    new_content = re.sub(r'\d{2,4}年\d{1,2}月','',new_content)
    new_content = re.sub(r'\d{2,4}年','',new_content)
    new_content = re.sub(r'\d{1,2}月\d{1,2}日','',new_content)
    new_content = re.sub(r'\d{1,2}月','',new_content)
    new_content = re.sub(r'\d{1,2}日','',new_content)
    new_content = re.sub(r'\d{2}:\d{2}:\d{2}','',new_content)
    new_content = re.sub(r'\d{2}:\d{2}','',new_content)
    # remove the punctuation marks
    new_content = re.sub(r'[%s]+' %punctuation,' ',new_content)
    # remove the , and .
    new_content = new_content.replace(',',' ')
    new_content = new_content.replace('.',' ')
    new_content = new_content.replace('(',' ')
    new_content = new_content.replace(')',' ')
    new_content = new_content.replace('=',' ')
    new_content = new_content.replace('?',' ')
    new_content = new_content.replace('/',' ')
    new_content = new_content.replace(':',' ')
    new_content = new_content.replace('-',' ')

    # remove the number
    new_content = re.sub(r'\d+','',new_content)

    return new_content

In [49]:
# info-level
info['fulltext'] = info['fulltext'].apply(lambda x:re_content(x))

In [53]:
# cmt-level
cmt['quote_content'].fillna(' ',inplace=True)
cmt['cmt_content'].fillna(' ',inplace=True)
# replace'此回覆已被刪除' with ' '
cmt['quote_content'] = cmt['quote_content'].apply(lambda x:x.replace('此回覆已被刪除',' '))
cmt['cmt_content'] = cmt['cmt_content'].apply(lambda x:x.replace('此回覆已被刪除',' '))
# clean the data
cmt['quote_content'] = cmt['quote_content'].apply(lambda x:re_content(x))
cmt['cmt_content'] = cmt['cmt_content'].apply(lambda x:re_content(x))

### 分词处理

In [50]:
CyberCan = pd.read_excel('./CyberCan.xlsx')
for i in range(len(CyberCan)):
    jieba.add_word(str(CyberCan['Words'][i]),freq=CyberCan['Frequency'][i])

jieba.add_word('Hong Kong',freq=100000)
jieba.add_word('九七',freq=20000)
jieba.add_word('等於',freq=20000)
jieba.add_word('由己',freq=20000)

In [51]:
immigration_dict = pd.read_excel('../pre_describe/immigration_dictionary_v1_0527.xlsx',skiprows=2,usecols=['Word'])
# add the words in the dictionary to jieba
for i in range(len(immigration_dict)):
    if immigration_dict['Word'][i] not in CyberCan['Words'].values:
        jieba.add_word(immigration_dict['Word'][i],freq=18883)

In [52]:
# from collections import Counter

stop_words = [' ','的','嘅','在','有','是','和','係','我','及','都','但','他','就','與','喺','為','對','要','到','而','了','被','你','會','又','同','或','也','亦','她','佢']

# # check the high frequency words and tune the stop words
# info_content_seg = []
# for i in info['fulltext']:
#     seg = jieba.lcut(i)
#     info_content_seg.append(seg)

# contents_clean = []
# for wl in info_content_seg:
#     wl_clean = []
#     for w in wl:
#         if w not in stop_words:
#             wl_clean.append(w)
#     contents_clean.append(wl_clean)

# all_words = []
# for wl in contents_clean:
#     all_words += wl

# counter = Counter(all_words)
# counter.most_common(20)

### 正式分词

In [53]:
def seg_words(text):
    seg = jieba.lcut(text)
    new_text = ' '.join(seg)
    return new_text

In [54]:
info['fulltext'] = info['fulltext'].apply(lambda x:seg_words(x))
# cmt['quote_content'] = cmt['quote_content'].apply(lambda x:seg_words(x))
# cmt['cmt_content'] = cmt['cmt_content'].apply(lambda x:seg_words(x))

In [55]:
info_hotness = pd.read_csv('../data0629/info_hotness_immigration.csv')
# cmt_hotness = pd.read_csv('../data0629/cmt_hotness_immigration.csv')

In [56]:
# combine the info and info_hotness
info = pd.merge(info,info_hotness,on='info_id',how='left')
# combine the cmt and cmt_hotness
# cmt = pd.merge(cmt,cmt_hotness,on='cmt_id',how='left')

In [58]:
info = info[['platform_x','publish_time','info_id','user_id','title','fulltext','contain_url','view_count', 'like_count', 'dislike_count',
       'cmt_count', 'repost_count']]
info.rename(columns={'platform_x':'platform'},inplace=True)
info['view_count'] = info['view_count'].fillna(0).astype(int)
info['like_count'] = info['like_count'].fillna(0).astype(int)
info['dislike_count'] = info['dislike_count'].fillna(0).astype(int)
info['cmt_count'] = info['cmt_count'].fillna(0).astype(int)
info['repost_count'] = info['repost_count'].fillna(0).astype(int)
info.to_csv('../data0629/info_processed_1029.csv',index=False,encoding='utf-8-sig')

In [70]:
cmt = cmt[['platform_x', 'info_id', 'cmt_time', 'cmt_id', 'cmt_user_id','cmt_tgt_user_id',
       'have_quote', 'quote_content', 'cmt_content', 'cmt_at_user', 'cmt_contain_url', 
       'like_count', 'dislike_count', 'reply_count','repost_count']]
cmt.rename(columns={'platform_x':'platform'},inplace=True)
cmt['cmt_id'] = cmt['cmt_id'].astype(str)
# turn the cmt_user_id to int and ignore the nan, then save as string
cmt['cmt_user_id'] = cmt['cmt_user_id'].dropna().astype(int)
cmt['cmt_user_id'] = cmt['cmt_user_id'].astype(str)
cmt['cmt_user_id'] = cmt['cmt_user_id'].apply(lambda x:x.replace('nan',''))

cmt['have_quote'] = cmt['have_quote'].fillna(0).astype(int)
cmt['like_count'] = cmt['like_count'].fillna(0).astype(int)
cmt['dislike_count'] = cmt['dislike_count'].fillna(0).astype(int)
cmt['reply_count'] = cmt['reply_count'].fillna(0).astype(int)
cmt['repost_count'] = cmt['repost_count'].fillna(0).astype(int)

cmt.to_csv('../data0629/cmt_processed1029.csv',index=False,encoding='utf-8-sig')

### LIWC结果合并

In [61]:
info = pd.read_csv('../data0629/info_processed_1029.csv',encoding='utf-8-sig')
info_LIWC1 = pd.read_csv('../data0629/info_processed_LIWC1_1030.csv')
info_LIWC2 = pd.read_csv('../data0629/info_processed_LIWC2_1030.csv',usecols=['info_id','WC', 'posemo', 'negemo', 'anx', 'anger','sad','swear'])

info = pd.merge(info,info_LIWC1,on='info_id',how='left')
info = pd.merge(info,info_LIWC2,on='info_id',how='left')

In [62]:
info['Stance'] = info['Localists']-info['支持大陸']

# normalize the stance to -1~1
info['Stance'] = info['Stance'].apply(lambda x: x/(info['Localists'].max()-info['Localists'].min()))

# Stage division
# Stage 0: before 2020-6-30
# Stage 1: 2020-6-30 to 2021-1-5
# Stage 2: 2021-1-6 to 2021-2-7
# Stage 3: 2021-2-8 to 2021-6-16
# Stage 4: 2021-6-17 to 2021-8-12
# Stage 5: 2021-8-13 to 2021-10-28
# Stage 6: 2021-10-29 to 2022-2-20
# Stage 7: 2022-2-21 to present
info['stage'] = info['publish_time'].apply(lambda x:0 if x<'2020-06-30' else 1 if x<'2021-01-06' else 2 if x<'2021-02-08' else 3 if x<'2021-06-17' else 4 if x<'2021-08-13' else 5 if x<'2021-10-29' else 6 if x<'2022-02-21' else 7)
info.to_csv('../data0629/info_afterLIWC_1030.csv',index=False,encoding='utf-8-sig')

In [68]:
info_stm = info.copy()

# log trans the WC
info_stm['WC'] = info_stm['WC'].apply(lambda x:np.log(x))

# senti
info_stm['senti'] = info_stm['posemo']-info_stm['negemo']
info_stm['senti'] = info_stm['senti'].apply(lambda x: x/(info_stm['posemo'].max()))

info_stm = info_stm[['platform', 'info_id', 'stage', 'fulltext', 'WC','senti','anx','anger','sad','swear','攻擊型','Stance']]
info_stm = info_stm[info_stm['stage']>0]
info_stm['stage'] = info_stm['stage'].apply(lambda x:'Stage'+str(x))
info_stm.to_csv('../data0629/info_stm_1030.csv',index=False, encoding='utf-8')

In [69]:
# save to xlsx: 保存为xlxs最好
import xlsxwriter

info_stm.to_excel('../data0629/info_stm_1030.xlsx',index=False, engine='xlsxwriter')

In [70]:
# info_copy = info[['platform','info_id','fulltext']]
# info_copy.rename(columns={'fulltext':'text'},inplace=True)
# info_copy['level'] = 'info'
# cmt_copy = cmt[['platform','info_id','cmt_content']]
# cmt_copy.rename(columns={'cmt_content':'text'},inplace=True)
# cmt_copy['level'] = 'cmt'

# # combine the info and cmt
# stm_data = pd.concat([info_copy,cmt_copy],axis=0)

In [71]:
# stm_data.to_csv('../data0629/stm_data1027.csv',index=False,encoding='utf-8')